In [1]:
import pandas as pd
import numpy as np

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate

In [2]:
df = pd.read_excel('RAG/chunks.xlsx')

In [21]:
def create_questions(text):
    model = ChatOpenAI(model='gpt-4o-mini-2024-07-18', max_tokens=1024, api_key='sk-WjFjU5gAjCTONWmXtUsUksWsO6GHbJvo', base_url = 'https://api.proxyapi.ru/openai/v1', temperature = 0.1)
    
    sys_prompt = """
        Ситуация: пользователь задает вопросы ИИ-ассистенту, в знаниях которого есть образовательный контент с курса о методике OKR (Objectives and Key Results)
        Внимательно прочитай текст и придумай к нему 3 вопроса, на которые в этом тексте есть ответ. Вот сам текст \n{text}\n.
        1. Придумай открытые вопросы, на которые нельзя ответить просто да или нет.​
        2. Всегда формулируйте очень краткие и четкие вопросы (менее 15 слов), которые пользователь может задать ИИ-ассистенту, чтобы получить ответ основываясь на этом тексте.
        3. НЕ создавай одинаковые или схожие вопросы.​
        4. САМОЕ ВАЖНОЕ - создавай только ТЕ вопросы, на которые в тексте ЕСТЬ ОТВЕТ.
        4. Выдай ответ в JSON словаре, где каждый ключ имеет в названии слово question и порядковый номер вопроса.
        """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", sys_prompt),
        ("human", "{text}"),
    ])
        
    # Define the chain using LCEL Соединение модели и промпта через |
    class ThreeExtraQuestions(BaseModel):
        question1: str = Field(description="Первый вопрос")
        question2: str = Field(description="Второй вопрос")
        question3: str = Field(description="Третий вопрос")
    
    # Set up a parser + inject instructions into the prompt template.
    parser = JsonOutputParser(pydantic_object=ThreeExtraQuestions)
    #а Персональные OKR - хороший инструмент целеполагания сотрудников, рекомендуется при начале внедрения OKR?
    chain = prompt | model | parser
    inv = chain.invoke({'text':text})
    return inv

In [35]:
questions_list = []
num=0
for i in df['Text']:
    num+=1
    q_l = create_questions(i)
    questions_list.append(q_l)
    print(num)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136


In [32]:
df['Text'][2]

'Вторая, связанная с первой проблема – мало кто в компании знает эти стратегические приоритеты. \nВ MIT исследовали, как по 4-уровневой типовой иерархии компании падает знание стратегических приоритетов в зависимости от движения сверху вниз.\nТоп-менеджмент знает порядка 50%. Для миддлов и ниже этот процент падает до 15-20, при этом последние обычно отвечают за исполнение стратегии. \nКак можно говорить об успешной реализации стратегии, если мы видим только ее кусочек?\n'

In [43]:
for_col = []
for dct in questions_list:
    for_col.append(' '.join(i for i in dct.values()))

In [45]:
df['questions'] = for_col

In [47]:
df['NewText'] = df['questions'] + '\n\n' + df['Text']

In [50]:
print(df['NewText'][0])

С какой темы начинается видео о методике OKR? Что будет рассмотрено после вопроса о необходимости изменений в целеполагании? Какие примеры будут разобраны в видео о методике OKR?

В этом первом видео мы разберем определение и критерии OKR – то есть целей и ключевых результатов.
А начнем мы с вопроса “зачем вообще что-то менять целеполагании?”, затем продолжим вопросом “что такое OKR?”, и разберем примеры хороших и плохих целей и ключевых результатов.


In [51]:
df.to_excel('new_chunks.xlsx')